# Run This Cell First #

This cell will import all the necessary libraries and also is where all of our functions are defined.

In [57]:
import numpy as np
from scipy import sparse
import time as time

def import_data(path):
    '''This will import the data from the text files nad output the data as a scipy sparse matrix.
    
    Inputs: path
        path: The file path the data. This should be a txt file in the form given in class.
        
    Outputs: M
        M: This is a scipy sparse matrix corresponding to the data. This will be an adjacency matrix.
    
    '''

    f = open(path,"r")
    phase1 = set()

    for line in f.readlines():
        x = line.split()
        x = tuple([int(i) for i in x])
        phase1.add(x)
    
    n,m = max([i[0] for i in phase1]), max([i[1] for i in phase1])
    
    M = np.zeros((n,m), dtype = int)
    
    for x in phase1:
        M[x[0]-1][x[1]-1] = 1
    
    M = sparse.csr_matrix(M)
    
    return M

def greedy(M, return_indices = False):
    '''Apply the greedy algorithm on the given sparse adjacency matrix M. This will output the number of poles selected. 
    
    Inputs: M, return_indices = False
        M: A scipy sparse matrix. Here, let the rows represent the meter number and the columns represent the receiver location. If the pole in row i is covered by a reciever in location j, then we get a value of 1 in row i, column j. Otherwise, we have a value of 0.
        return_indices: If true, the indices of the selected poles will be returned
        
    Outputs: numpoles, indices
        numpoles: This is the number of poles selected.
        indices: The set of indices of the selected poles.
        
        '''
    
    #Dimensions of M
    m,n = M.shape
    
    #Sparse col vector of all 1s
    col1 = [1 for i in range(n)]
    col1 = sparse.csr_matrix(col1, dtype = int).transpose()
    
    
    indices = set()
     
    
    while True:
        
        #Sparse row vector of all 1s
        row1 = [1 for i in range(M.shape[0])]
        row1 = sparse.csr_matrix(row1, dtype = int)
        
        #Count the number of meters covered by each receiver
        try:
            A = row1@M
            #Find the maximum number of remaining receivers to cover
            x = A.max()
        except:
            x = 0
        
        #If there are no remaining meters, break the loop
        if x == 0:
            break
        
        
        #Find which receiver covers the most meters
        bestrec = A.argmax()
        indices.add(bestrec)
        
                                 
        #Remove all meters covered by x
        
        #Indicator vector for receiver
        recvec = np.zeros(n)
        recvec[bestrec] = 1
        recvec = sparse.csr_matrix(recvec, dtype = int).transpose()
        
        #This will tell us which meters are covered by x
        covered = M@recvec
        covered = covered.toarray()[:,0]
        
        #Any now, we can remove the rows
        mask = (1-covered).astype(bool)
        try:
            M = M[mask]
        except:
            pass
        
    numpoles = len(indices)
    if return_indices:
        return numpoles,indices
    else:
        return numpoles
    
def modified_greedy(M, w1=1, w2=1,return_indices = False):
    '''Apply the modified greedy algorithm on the given sparse adjacency matrix M. This will output the number of poles selected. 
    
    Inputs: M, return_indices = False, w1=1, w2=1
        M: A scipy sparse matrix. Here, let the rows represent the meter number and the columns represent the receiver location. If the pole in row i is covered by a reciever in location j, then we get a value of 1 in row i, column j. Otherwise, we have a value of 0.
        w1: A weight for score1 in the sum
        w2: A weight for score2 in the sum
        return_indices: If true, the indices of the selected poles will be returned
        
    Outputs: numpoles, indices
        numpoles: This is the number of poles selected.
        indices: The set of indices of the selected poles.
        
        '''
    
    #Dimensions of M
    m,n = M.shape
    
    #Sparse col vector of all 1s
    col1 = [1 for i in range(n)]
    col1 = sparse.csr_matrix(col1, dtype = int).transpose()
    
    indices = set()
     
    
    while True:
        
        #Score 
        
        #Find the hardest to cover meters
        try:
            hardtocov = M@col1
            mincov = hardtocov.min()
        except:
            break
        
        #remove rows of zeros
        while mincov == 0:
            row = np.argmin(mincov)
            mask = np.ones(M.shape[0],dtype = bool)
            mask[row] = 0
            
            M = M[mask]
            
            hardtocov = M@col1
            mincov = hardtocov.min()
        
        hardtocov = hardtocov.toarray()[:,0]
        hardtocov = np.array([1 for i in hardtocov if i == mincov])
        try:
            mask = np.array(hardtocov).astype(bool)
        except:
            break
        
        ScoringM = M[mask]
        
        #Sparse row vector of all 1s
        row1 = [1 for i in range(M.shape[0])]
        row1 = sparse.csr_matrix(row1, dtype = int)
        
        row1scoring = [1 for i in range(ScoringM.shape[0])]
        row1scoring = sparse.csr_matrix(row1scoring, dtype = int)

        
        #Count the number of meters covered by each receiver
        try:
            A = row1@M
            B = row1scoring@ScoringM
            
            #Here, we will take score 1 + score 2 with weights w1 and 12
            C = w1*B + w2*A.multiply(B) 
            #Find the maximum number of remaining receivers to cover
            x = C.max()
        except:
            x = 0
        
        #If there are no remaining meters, break the loop
        if x == 0:
            break
        
        
        #Find which receiver covers the most meters
        bestrec = C.argmax()
        indices.add(bestrec)
        
                                 
        #Remove all meters covered by x
        
        #Indicator vector for receiver
        recvec = np.zeros(n)
        recvec[bestrec] = 1
        recvec = sparse.csr_matrix(recvec, dtype = int).transpose()
        
        #This will tell us which meters are covered by x
        covered = M@recvec
        covered = covered.toarray()[:,0]
        
        #Any now, we can remove the rows
        mask = (1-covered).astype(bool)
        try:
            M = M[mask]
        except:
            pass
        
    numpoles = len(indices)
    if return_indices:
        return numpoles,indices
    else:
        return numpoles
    
def preprocess(M, k=-1, l=-1):
    '''This will perform the preprocessing step to the given matrix M
    
    inputs: M, k, l
        M: M: A scipy sparse matrix. Here, let the rows represent the meter number and the columns represent the receiver location. If the pole in row i is covered by a reciever in location j, then we get a value of 1 in row i, column j. Otherwise, we have a value of 0.
        k: This is the number of rows to check in the row removal step
        l: This is the number of columns to check in the column removal step
    
    outputs: M, indices
        M: The preprocessed matrix. This will be a modification of M and will still be a scipy sparse matrix
        indices: This will be a list of tuples containing the selected and removed poles. If pole i was selected,this will be indicated as (i,0). If pole i is removed in preprocessing, this will be indicated as (i,1). The list will add poles in the order they are removed.
    '''
    if k==-1:
        k = M.shape[0]
    if l==-1:
        l = M.shape[1]
    indices = []
    
    col1 = [1 for i in range(M.shape[1])]
    col1 = sparse.csr_matrix(col1, dtype = int).transpose()
    
    #Remove Singleton Rows
    
    #Find the hardest to cover meters
    try:
        hardtocov = M@col1
        hardtocov = hardtocov.toarray()
        mincov = hardtocov.min()
    except:
        return M,indices
    
    
    while mincov<=1:
        try:
            row = mincov.argmin()
        except:
            break
            
            #Remove a row of all zeros
        if mincov == 0:
            mask = np.ones(M.shape[0],dtype = bool)
            try:
                mask[row] = 0
            except:
                break
                
            M = M[mask]

            hardtocov = M@col1
            try:
                mincov = hardtocov.min()
            except:
                break

            
        #Select the pole that covers a single 1 column
        else:
            #Find which pole covers the hard to cover meter
            indicator = np.zeros(M.shape[0],dtype = bool)
            indicator[row] = 1
            indicator = sparse.csr_matrix(indicator, dtype = int)
            indicatorsum = indicator@M
            colind = np.argmax(indicatorsum)
            indicatorc = np.zeros(M.shape[1],dtype = bool)
            indicatorc[colind] = 1
                
            indices += [(colind,0)]
            M = M[indicatorc]
                
            hardtocov = M@col1
            mincov = hardtocov.min()
    
    #Remove Rows Contained in Other Rows
    count = 0
    
    while count < min(k,M.shape[0]):
        
        #This is the dummy matrix
        A = sparse.csc_matrix(M)
        
        #This will find which poles cover a given meter
        rowindicator = np.zeros(A.shape[0],dtype=bool)
        rowindicator[count] = 1
        rowindicator = sparse.csc_matrix(rowindicator, dtype=int)
        
        #Contains will be a vector that is 1 for all poles that do not cover our meter, 0 for all poles that do
        contains = rowindicator@A
        contains = contains.toarray()
        contains = np.array(contains,dtype=bool).transpose()[:,0]
        contains = 1-contains
        
        #This will remove the columns from A that correspond to poles that cover our meter
        A = A[:,contains]
        A = sparse.csr_matrix(A)
        
        #Now we remove redundant rows
        col1 = np.ones(A.shape[1],dtype=int)
        col1 = sparse.csr_matrix(col1).transpose()
        
        checks = A@col1
        checks = checks.toarray()[:,0]
        try:
            rem = np.ones(checks.size, dtype=bool)
            
        except:
            count += 1
            continue
        
        try:
            for i in range(checks.size):
                if i != count and checks[i] == 0:
                    rem[count] = 0
                    break
            M = M[rem]
        except:
            pass
        count += 1
    
    #Remove Columns Contained in Other Columns
    count = 0
    
    while count < min(l,M.shape[1]):
        
        #This is the dummy matrix
        A = sparse.csr_matrix(M)
        
        #This will find which meters cover a given receiver
        colindicator = np.zeros(A.shape[1],dtype=bool)
        try:
            colindicator[count] = 1
            colindicator = sparse.csc_matrix(colindicator, dtype=int).transpose()
            
            #This vector will have a 1 for all meters not covered by our receiver, and a 0 for all meters that are covered
            contains = A@colindicator
            contains = contains.toarray()
            contains = np.array(contains,dtype=bool)[:,0]
            contains = 1-contains
        except:
            count+=1
            continue

        #This will remove the rows from A that are covered by our receiver
        try:
            A = A[contains]
            A = sparse.csc_matrix(A)
        except:
            break
        
        if A.shape[0] == 0:
            count += 1
            continue
        
        #Now, we remove redundant columns
        col1 = np.ones(A.shape[1],dtype=int)
        col1 = sparse.csr_matrix(col1).transpose()
        
        checks = A@col1
        checks = checks.toarray()[:,0]
        
        try:
            rem = np.ones(checks.size, dtype=bool)
        except:
            count+=1
            continue
        
        
        try:
            for i in range(checks.shape):
                if i != count and checks[i]==0:
                    rem[i] = 0
                    indices += [(i,1)]
            M = sparse.csc_matrix(M)
            M = M[:,rem]
            M = sparse.csr_matrix(M)
            
        except:
            pass
        

        count += 1
    return M,indices

def getind(indices):
    '''Get the selected columns from the indices list
    
    Inputs: indices
        A list of tuples corresponding to the indices of the removed and selected columns
        
    Outputs: selected
        A list of integers corresponding to the selected columns'''
    
    

def greedyp(M, return_indices = False, p=1, k=-1, l=-1):
    '''Apply the greedy algorithm on the given sparse adjacency matrix M with preprocessing. This will output the number of poles selected. 
    
    Inputs: M, return_indices = False, p=1
        M: A scipy sparse matrix. Here, let the rows represent the meter number and the columns represent the receiver location. If the pole in row i is covered by a reciever in location j, then we get a value of 1 in row i, column j. Otherwise, we have a value of 0.
        return_indices: If true, the indices of the selected poles will be returned
        p: This is the number of iterations between preprocessing steps
        k: This is the number of rows to check in the row removal step
        l: This is the number of columns to check in the column removal step
        
    Outputs: numpoles, indices
        numpoles: This is the number of poles selected.
        indices: The set of indices of the selected poles.
        
        '''
    
    #Dimensions of M
    m,n = M.shape
    
    #Sparse col vector of all 1s
    col1 = [1 for i in range(n)]
    col1 = sparse.csr_matrix(col1, dtype = int).transpose()
    
    
    indices = []
    iteration = 0
    
    while True:
        
        if iteration%p == 0:
            M,i = preprocess(M, k, l)
            indices += i
        
        #Sparse row vector of all 1s
        row1 = [1 for i in range(M.shape[0])]
        row1 = sparse.csr_matrix(row1, dtype = int)
        
        #Count the number of meters covered by each receiver
        try:
            A = row1@M
            #Find the maximum number of remaining receivers to cover
            x = A.max()
        except:
            x = 0
        
        #If there are no remaining meters, break the loop
        if x == 0:
            break
        
        
        #Find which receiver covers the most meters
        bestrec = int(A.argmax())
        indices += [(bestrec,0)]
        
                                 
        #Remove all meters covered by x
        
        #Indicator vector for receiver
        recvec = np.zeros(n)
        recvec[bestrec] = 1
        recvec = sparse.csr_matrix(recvec, dtype = int).transpose()
        
        #This will tell us which meters are covered by x
        covered = M@recvec
        covered = covered.toarray()[:,0]
        
        #Any now, we can remove the rows
        mask = (1-covered).astype(bool)
        try:
            M = M[mask]
        except:
            pass
        
        iteration += 1
    
    #only include selected indices in final count
    countl = [i for i in indices if i[1] == 0]
    numpoles = len(indices)
    
    if return_indices:
        selected = getind(indices)
        return numpoles,selected
    else:
        return numpoles
    
def modified_greedyp(M, w1=1, w2=1, p=1, k=-1, l=-1,return_indices = False):
    '''Apply the modified greedy algorithm on the given sparse adjacency matrix M. This will output the number of poles selected. 
    
    Inputs: M, return_indices = False, w1=1, w2=1
        M: A scipy sparse matrix. Here, let the rows represent the meter number and the columns represent the receiver location. If the pole in row i is covered by a reciever in location j, then we get a value of 1 in row i, column j. Otherwise, we have a value of 0.
        w1: A weight for score1 in the sum
        w2: A weight for score2 in the sum
        k: This is the number of rows to check in the row removal step
        l: This is the number of columns to check in the column removal step
        return_indices: If true, the indices of the selected poles will be returned
        
    Outputs: numpoles, indices
        numpoles: This is the number of poles selected.
        indices: The set of indices of the selected poles.
        
        '''
    
    #Dimensions of M
    m,n = M.shape
    
    #Sparse col vector of all 1s
    col1 = [1 for i in range(n)]
    col1 = sparse.csr_matrix(col1, dtype = int).transpose()
    
    indices = []
    iteration = 0
     
    
    while True:
        
        if iteration%p == 0:
            M,i = preprocess(M,k,l)
            indices += i
        
        #Score 
        
        #Find the hardest to cover meters
        try:
            hardtocov = M@col1
            mincov = hardtocov.min()
        except:
            break
        
        #remove rows of zeros
        while mincov == 0:
            row = np.argmin(mincov)
            mask = np.ones(M.shape[0],dtype = bool)
            mask[row] = 0
            
            M = M[mask]
            
            hardtocov = M@col1
            mincov = hardtocov.min()
        
        hardtocov = hardtocov.toarray()[:,0]
        hardtocov = np.array([1 for i in hardtocov if i == mincov])
        try:
            mask = np.array(hardtocov).astype(bool)
        except:
            break
        
        ScoringM = M[mask]
        
        #Sparse row vector of all 1s
        row1 = [1 for i in range(M.shape[0])]
        row1 = sparse.csr_matrix(row1, dtype = int)
        
        row1scoring = [1 for i in range(ScoringM.shape[0])]
        row1scoring = sparse.csr_matrix(row1scoring, dtype = int)

        
        #Count the number of meters covered by each receiver
        try:
            A = row1@M
            B = row1scoring@ScoringM
            
            #Here, we will take score 1 + score 2 with weights w1 and 12
            C = w1*B + w2*A.multiply(B) 
            #Find the maximum number of remaining receivers to cover
            x = C.max()
        except:
            x = 0
        
        #If there are no remaining meters, break the loop
        if x == 0:
            break
            
        
        #Find which receiver covers the most meters
        bestrec = C.argmax()
        indices += [(bestrec,0)]
        
                                 
        #Remove all meters covered by x
        
        #Indicator vector for receiver
        recvec = np.zeros(n)
        recvec[bestrec] = 1
        recvec = sparse.csr_matrix(recvec, dtype = int).transpose()
        
        #This will tell us which meters are covered by x
        covered = M@recvec
        covered = covered.toarray()[:,0]
        
        #Any now, we can remove the rows
        mask = (1-covered).astype(bool)
        try:
            M = M[mask]
        except:
            pass
        
        iteration += 1
        
    numpoles = len(indices)
    if return_indices:
        return numpoles,indices
    else:
        return numpoles

# Run This Cell Second  #

This cell will import the phase1 and cap360 data.

In [49]:
phase1 = import_data('phase1.txt')
cap360 = import_data('cap360.txt')

# Test Cases #

Each of the cells below is a different test case. The specific algorithm to be tested is stated above the cell.

## Greedy Algorithm ##



In [50]:
phase1tim = 0
cap360tim = 0

phase1tim -= time.time()
phase1poles = greedy(phase1)
phase1tim += time.time()

cap360tim -= time.time()
cap360poles = greedy(cap360)
cap360tim += time.time()

print("The number of poles in phase1 using the greedy algorithm is: " + str(phase1poles))
print("The greedy algorithm took "+str(phase1tim)+" s to run")
print("The number of poles in cap360 using the greedy algorithm is: " + str(cap360poles))
print("The greedy algorithm took "+str(cap360tim)+" s to run")

The number of poles in phase1 using the greedy algorithm is: 24
The greedy algorithm took 0.021967172622680664 s to run
The number of poles in cap360 using the greedy algorithm is: 625
The greedy algorithm took 0.9469187259674072 s to run


## Modified Greedy Algorithm ##



In [51]:
phase1tim = 0
cap360tim = 0

phase1tim -= time.time()
phase1poles = modified_greedy(phase1, w1=0.2, w2=0.1)
phase1tim += time.time()

cap360tim -= time.time()
cap360poles = modified_greedy(cap360, w1=1.7, w2=0.1)
cap360tim += time.time()

print("The number of poles in phase1 using the modified greedy algorithm is: " + str(phase1poles))
print("The modified greedy algorithm took "+str(phase1tim)+" s to run")
print("The number of poles in cap360 using the modified greedy algorithm is: " + str(cap360poles))
print("The modified greedy algorithm took "+str(cap360tim)+" s to run")

The number of poles in phase1 using the modified greedy algorithm is: 23
The modified greedy algorithm took 0.10683798789978027 s to run
The number of poles in cap360 using the modified greedy algorithm is: 613
The modified greedy algorithm took 2.2577102184295654 s to run


## Greedy Algorithm With Preprocessing



In [60]:
phase1tim = 0
cap360tim = 0

phase1tim -= time.time()
phase1poles = greedyp(phase1, p=100)
phase1tim += time.time()

cap360tim -= time.time()
cap360poles = greedyp(cap360, p=100)
cap360tim += time.time()

print("The number of poles in phase1 using the greedy algorithm with preprocessing is: " + str(phase1poles))
print("The greedy algorithm with preprocessing took "+str(phase1tim)+" s to run")
print("The number of poles in cap360 using the greedy algorithm with preprocessing is: " + str(cap360poles))
print("The greedy algorithm with preprocessing took "+str(cap360tim)+" s to run")

The number of poles in phase1 using the greedy algorithm with preprocessing is: 19
The greedy algorithm with preprocessing took 0.1528027057647705 s to run
The number of poles in cap360 using the greedy algorithm with preprocessing is: 308
The greedy algorithm with preprocessing took 41.24045753479004 s to run


## Modified Greedy Algorithm With Preprocessing



In [61]:
phase1tim = 0
cap360tim = 0

phase1tim -= time.time()
phase1poles = modified_greedyp(phase1,p=100, w1=0, w2=1)
phase1tim += time.time()

cap360tim -= time.time()
cap360poles = modified_greedyp(cap360,p=100, w1=3, w2=1)
cap360tim += time.time()

print("The number of poles in phase1 using the moified greedy algorithm with preprocessing is: " + str(phase1poles))
print("The modified greedy algorithm with preprocessing took "+str(phase1tim)+" s to run")
print("The number of poles in cap360 using the moified greedy algorithm with preprocessing is: " + str(cap360poles))
print("The modified greedy algorithm with preprocessing took "+str(cap360tim)+" s to run")

The number of poles in phase1 using the moified greedy algorithm with preprocessing is: 19
The modified greedy algorithm with preprocessing took 0.18288445472717285 s to run
The number of poles in cap360 using the moified greedy algorithm with preprocessing is: 323
The modified greedy algorithm with preprocessing took 45.53054618835449 s to run


## Full Attempt on Cap360



In [56]:
tim1 = 0
tim2 = 0

tim1 -= time.time()
poles1 = greedyp(cap360)
tim1 += time.time()

tim2 -= time.time()
poles2 = modified_greedyp(cap360, w1=1.7, w2=0.1)
tim2 += time.time()

print("The number of poles in the full greedy algorithm with preprocessing is: " + str(poles1))
print("The full greedy algorithm with preprocessing took "+str(tim1)+" s to run")
print("The number of poles in the full modified greedy algorithm is: " + str(poles2))
print("The full modified greedy algorithm with preprocessing took "+str(tim2)+" s to run")

The number of poles in the full greedy algorithm with preprocessing is: 12
The full greedy algorithm with preprocessing took 98.44094276428223 s to run
The number of poles in the full modified greedy algorithm is: 12
The full modified greedy algorithm with preprocessing took 95.98308086395264 s to run
